<div style="color:white;
            display:fill;
            border-radius:20px;
            background-color:green;
            font-size:200%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 10px;
               margin:0px auto 0px auto;
              color:white;
              text-align:center;'>
       &nbsp;Depth Prediction with MiDaS
    </h1>
    </div>

## This the code for predicting the weight of waste present in the image.

## just run the cells 1 by 1 and wait for the folders to form in the storage section.

## Insert your images and yolo format annotaion file in anno_folder

In [20]:
import cv2
import torch
import urllib.request
import numpy as np
import os
import matplotlib.pyplot as plt
from torchvision import transforms
import time
from PIL import Image
import scipy.stats as stats

In [ ]:
# Create 2 new folders
os.makedirs('/content/annotated_files')

os.makedirs('/content/midas_images')

# --> The annotated_files folder can store the Input images and annotation files of the images. Make sure to keep the name of image and annotation file same.

# --> The midas_images folder is system generated and is for the system only. Do not modify this folder

In [ ]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.1 MB/s eta 0:00:00


In [ ]:
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt
100%|██████████| 1.28G/1.28G [00:20<00:00, 65.6MB/s]
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS

In [ ]:
# Path to the folder containing YOLO annotated files
anno_folder = '/content/annotated_files'

# Path to the Midas depth image
midas_img_path = '/content/midas_images'

# Create an empty dictionary to store the results
results = {}

# Define the density of the material in cm^3/g (hand crafted)
density = {'Aluminium foil': 2.7,'Battery': 1.5,'Aluminium blister pack':2.71,'Carded blister pack': 2.4,'Other plastic bottle': 1.6,'Clear plastic bottle': 1,'Glass bottle': 3.2,'Plastic bottle cap': 0.92,'Metal bottle cap': 1.3,'Broken glass': 3.2,'Food Can': 3.6,'Aerosol': 2,'Drink can': 1,'Toilet tube': 2.2,'Other carton': 1.6,'Egg carton': 1.4,'Drink carton': 1.7,'Corrugated carton': 0.5,'Meal carton': 1.4,'Pizza box': 1.8,'Paper cup': 0.8,'Disposable plastic cup': 0.8,'Foam cup':0.6,'Glass cup': 2.2,'Other plastic cup': 0.8,'Food waste': 1.4,'Glass jar':2.4,'Plastic lid': 0.8,'Metal lid': 1.2,'Other plastic': 0.9,'Magazine paper': 0.3,'Tissues': 0.3,'Wrapping paper': 0.5,'Normal paper': 0.7,'Paper bag':0.7,'Plastified paper bag': 0.7,'Plastic film' : 1.2,'Six pack rings': 1.3,'Garbage bag': 4,'Other plastic wrapper' : 1.2,'Single-use carrier bag': 1.2,'Polypropylene bag': 1.3,'Crisp packet': 2.2,'Spread tub' : 3.4,'Tupperware' : 4.2,'Disposable food container':2.7,'Foam food container': 1.7,'Other plastic container': 1.7,'Plastic glooves': 2.2,'Plastic utensils': 1.9,'Pop tab': 0.4,'Rope & strings': 0.2,'Scrap metal' : 4.3,'Shoe' : 3.8,'Squeezable tube': 1.7,'Plastic straw' : 0.8,'Paper straw': 0.6,'Styrofoam piece' : 2.2,'Unlabeled litter': 2,'Cigarette': 4.2}

# Define a constant factor to scale the weight calculation (set according to comparitions of the results with the ground truth)
scale_factor = 0.001

# Initialize a variable to store the total weight of the image
total_weight = 0

# Loop through the annotated files in the folder
for filename in os.listdir(anno_folder):
    if filename.endswith('.jpg'):
        # Read input image
        img = cv2.imread(os.path.join(anno_folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        input_batch = transform(img).to(device)

        with torch.no_grad():
            prediction = midas(input_batch)

            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        output = prediction.cpu().numpy()
        print(filename)

        # Save output image
        save_filename = os.path.join(midas_img_path, os.path.splitext(filename)[0] + '_midas')
        plt.imsave(save_filename + '.png', output, cmap='gray')

batch_2-000039.jpg


In [ ]:
density = {i+1: density[k] for i, k in enumerate(density)}
print(density)

{1: 2.7, 2: 1.5, 3: 2.71, 4: 2.4, 5: 1.6, 6: 1, 7: 3.2, 8: 0.92, 9: 1.3, 10: 3.2, 11: 3.6, 12: 2, 13: 1, 14: 2.2, 15: 1.6, 16: 1.4, 17: 1.7, 18: 0.5, 19: 1.4, 20: 1.8, 21: 0.8, 22: 0.8, 23: 0.6, 24: 2.2, 25: 0.8, 26: 1.4, 27: 2.4, 28: 0.8, 29: 1.2, 30: 0.9, 31: 0.3, 32: 0.3, 33: 0.5, 34: 0.7, 35: 0.7, 36: 0.7, 37: 1.2, 38: 1.3, 39: 4, 40: 1.2, 41: 1.2, 42: 1.3, 43: 2.2, 44: 3.4, 45: 4.2, 46: 2.7, 47: 1.7, 48: 1.7, 49: 2.2, 50: 1.9, 51: 0.4, 52: 0.2, 53: 4.3, 54: 3.8, 55: 1.7, 56: 0.8, 57: 0.6, 58: 2.2, 59: 2, 60: 4.2}


In [26]:
density = {1: 2.7, 2: 1.5, 3: 2.71, 4: 2.4}
factor = 0.00001

class_weights = {}
class_weight_sum = 0

for filename in os.listdir(anno_folder):
    if filename.endswith('.txt'):
        # Load the annotation file
        anno_file = os.path.join(anno_folder, filename)
        with open(anno_file, 'r') as f:
            lines = f.readlines()

        # Extract the coordinates of all the annotated areas
        class_coords = {}
        img_file = os.path.join(anno_folder, filename.replace('.txt', '.jpg'))  # Assumes image file has the same name as label file, but with a .jpg extension
        midas_file = os.path.join(midas_img_path, filename.replace('.txt', '_midas.png'))  # Assumes midas file has the same name as label file, but with a _midas.png extension
        with Image.open(img_file) as img:
            img_w, img_h = img.size
        with open(os.path.join(anno_folder, filename), 'r') as f:
            lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line != '':
                class_id, x_center_norm, y_center_norm, w_norm, h_norm = [float(x) for x in line.split()]
                x_center, y_center, w, h = int(x_center_norm * img_w), int(y_center_norm * img_h), int(w_norm * img_w), int(h_norm * img_h)
                x1, y1, x2, y2 = x_center - w // 2, y_center - h // 2, x_center + w // 2, y_center + h // 2
                print(class_id, x1, y1, x2, y2)
            if class_id not in class_coords:
               class_coords[class_id] = []
               class_coords[class_id].append((x1, y1, x2, y2))

        # Get the mode of the pixel values for each annotated area
        for class_id, coords in class_coords.items():
            mode_values = []
            for coord in coords:
              with Image.open(midas_file) as midas_img:
                x1, y1, x2, y2 = coord
                cropped_img = midas_img.crop((x1, y1, x2, y2))
                cropped_arr = np.array(cropped_img)
                mode_val = stats.mode(cropped_arr, axis=None).mode[0]
                mode_values.append(mode_val)

            # Calculate class weight
            area = sum([(x2-x1)*(y2-y1) for (x1, y1, x2, y2) in coords])
            density_val = density.get(class_id, 1)  # If class density not available, assume 1
            class_weight = area * np.mean(mode_values) * density_val * factor
            class_weights[class_id] = class_weight
            class_weight_sum += class_weight

print("Class weights:", class_weights)
print("Class weight sum:", class_weight_sum, "grams")

12.0 1069 1403 1269 1623
58.0 1238 730 1316 806


<ipython-input-26-45243bf3305b>:41: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_val = stats.mode(cropped_arr, axis=None).mode[0]


Class weights: {12.0: 112.2, 58.0: 1.8376800000000002}
Class weight sum: 114.03768000000001 grams
